In [1]:
import ee
import geemap
import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm 
import time
from datetime import datetime

In [2]:
# geemap.set_proxy(port=7890) # set your proxy port
ee.Authenticate()
ee.Initialize(project='socd-liuziyan') # change your GEE project name

## import socd and AlphaEarth data

In [3]:
soc_data = ee.FeatureCollection('projects/socd-liuziyan/assets/1_final_socd_0_100cm')

embedding_dataset = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
embedding_2024 = embedding_dataset.filterDate('2024-01-01', '2025-01-01').mosaic()

## building and tuning the RF model

In [4]:
# Split the SOC data into training (80%) and validation (20%) sets
soc_data = soc_data.randomColumn(columnName='random', seed=42)
training_fc = soc_data.filter(ee.Filter.lt('random', 0.8))
validation_fc = soc_data.filter(ee.Filter.gte('random', 0.8))

# Sample the embedding image at the locations of the SOC data
training = embedding_2024.sampleRegions(
    collection=training_fc, properties=['socd_100cm'], scale=10, tileScale=16
)
test = embedding_2024.sampleRegions(
    collection=validation_fc, properties=['socd_100cm'], scale=10, tileScale=16
)

# Tune the number of trees in Random Forest
band_names = embedding_2024.bandNames()
numTreesList = ee.List.sequence(0, 200, 20)

def tune(num):
    num = ee.Number(num)
    model = ee.Classifier.smileRandomForest(
        numberOfTrees=num,
        variablesPerSplit=8,
        minLeafPopulation=5,
        bagFraction=0.8,
        seed=42
    ).setOutputMode('REGRESSION').train(
        features=training,
        classProperty='socd_100cm',
        inputProperties=band_names
    )
    preds = test.classify(model).map(
        lambda f: f.set(
            'residual', ee.Number(f.get('classification')).subtract(f.get('socd_100cm')),
            'sq_residual', ee.Number(f.get('classification')).subtract(f.get('socd_100cm')).pow(2)
        )
    )
    mse = ee.Number(preds.reduceColumns(ee.Reducer.mean(), ['sq_residual']).get('mean'))
    rmse = ee.Number(mse).sqrt()
    return ee.Feature(None, {'numberOfTrees': num, 'rmse': rmse})

tuning_fc = ee.FeatureCollection(numTreesList.map(tune))
best = tuning_fc.sort('rmse', True).first()
optimal_num_trees = ee.Number(best.get('numberOfTrees'))


In [5]:
print('optimal numberOfTrees =', ee.Number(optimal_num_trees).getInfo())

EEException: Classifier training failed: 'Invalid number of trees: 0'.

In [5]:
# Training the optimal model
optimal_model = ee.Classifier.smileRandomForest(
    numberOfTrees=160,
    variablesPerSplit=8,
    minLeafPopulation=5,
    bagFraction=0.8,
    seed=42
).setOutputMode('REGRESSION').train(
    features=training,
    classProperty='socd_100cm',
    inputProperties=band_names
)

In [6]:
# calculating r2 and rmse of train set and validation set

def compute_metrics(samples, label_prop, model):
    preds = samples.classify(model).map(
        lambda f: f.set(
            'residual', ee.Number(f.get('classification')).subtract(f.get(label_prop)),
            'sq_residual', ee.Number(f.get('classification')).subtract(f.get(label_prop)).pow(2)
        )
    )
    mse = ee.Number(preds.reduceColumns(ee.Reducer.mean(), ['sq_residual']).get('mean'))
    rmse = mse.sqrt()

    y_mean = ee.Number(samples.reduceColumns(ee.Reducer.mean(), [label_prop]).get('mean'))
    ss_res = ee.Number(preds.reduceColumns(ee.Reducer.sum(), ['sq_residual']).get('sum'))
    ss_tot = ee.Number(
        samples.map(lambda f: f.set('sq_total', ee.Number(f.get(label_prop)).subtract(y_mean).pow(2)))
               .reduceColumns(ee.Reducer.sum(), ['sq_total']).get('sum')
    )
    r2 = ee.Number(1).subtract(ss_res.divide(ss_tot))
    return ee.Dictionary({'rmse': rmse, 'r2': r2})


training_samp = training
validation_samp = test

train_metrics = compute_metrics(training_samp, 'socd_100cm', optimal_model)
val_metrics = compute_metrics(validation_samp, 'socd_100cm', optimal_model)

print('Train RMSE:', ee.Number(train_metrics.get('rmse')).getInfo())
print('Train R2  :', ee.Number(train_metrics.get('r2')).getInfo())
print('Valid RMSE:', ee.Number(val_metrics.get('rmse')).getInfo())
print('Valid R2  :', ee.Number(val_metrics.get('r2')).getInfo())

Train RMSE: 10.177687521179314
Train R2  : 0.7746242869241102
Valid RMSE: 13.939769614381124
Valid R2  : 0.5212747520400874


In [32]:
# 10-fold Cross-Validation

def k_fold_split(features, k):
    step = ee.Number(1).divide(k)
    thresholds = ee.List.sequence(0, ee.Number(1).subtract(step), step)
    features = features.randomColumn(seed=0)
    def split(th):
        th = ee.Number(th)
        training = features.filter(ee.Filter.Or(
            ee.Filter.lt('random', th),
            ee.Filter.gte('random', th.add(step))
        ))
        validation = features.filter(ee.Filter.And(
            ee.Filter.gt('random', th),
            ee.Filter.lte('random', th.add(step))
        ))
        return ee.Feature(None, {'training': training, 'validation': validation})
    return ee.FeatureCollection(thresholds.map(split))

folds = k_fold_split(soc_data, 10)

def fold_metrics(fold):
    fold = ee.Feature(fold)
    training_fc = ee.FeatureCollection(fold.get('training'))
    validation_fc = ee.FeatureCollection(fold.get('validation'))
    train_samp = embedding_2024.sampleRegions(collection=training_fc, properties=['socd_100cm'], scale=10, tileScale=16)
    test_samp = embedding_2024.sampleRegions(collection=validation_fc, properties=['socd_100cm'], scale=10, tileScale=16)

    model = ee.Classifier.smileRandomForest(
        numberOfTrees=160,
        variablesPerSplit=8,
        minLeafPopulation=5,
        bagFraction=0.8,
        seed=42
    ).setOutputMode('REGRESSION').train(
        features=train_samp,
        classProperty='socd_100cm',
        inputProperties=band_names
    )

    preds = test_samp.classify(model).map(
        lambda f: f.set(
            'residual', ee.Number(f.get('classification')).subtract(f.get('socd_100cm')),
            'sq_residual', ee.Number(f.get('classification')).subtract(f.get('socd_100cm')).pow(2)
        )
    )
    mse = ee.Number(preds.reduceColumns(ee.Reducer.mean(), ['sq_residual']).get('mean'))
    rmse = mse.sqrt()

    y_mean = ee.Number(test_samp.reduceColumns(ee.Reducer.mean(), ['socd_100cm']).get('mean'))
    ss_res = ee.Number(preds.reduceColumns(ee.Reducer.sum(), ['sq_residual']).get('sum'))
    ss_tot = ee.Number(
        test_samp.map(lambda f: f.set('sq_total', ee.Number(f.get('socd_100cm')).subtract(y_mean).pow(2)))
                .reduceColumns(ee.Reducer.sum(), ['sq_total']).get('sum')
    )
    r2 = ee.Number(1).subtract(ss_res.divide(ss_tot))
    return ee.Feature(None, {'rmse': rmse, 'r2': r2})

cv_results = ee.FeatureCollection(folds.map(fold_metrics))

mean_rmse = ee.Number(cv_results.reduceColumns(ee.Reducer.mean(), ['rmse']).get('mean'))
mean_r2 = ee.Number(cv_results.reduceColumns(ee.Reducer.mean(), ['r2']).get('mean'))

print('10-fold RMSEs:', cv_results.aggregate_array('rmse').getInfo())
print('10-fold R2s:', cv_results.aggregate_array('r2').getInfo())
print('Mean RMSE:', mean_rmse.getInfo())
print('Mean R2:', mean_r2.getInfo())

10-fold RMSEs: [9.450142620713088, 8.845552666125336, 11.268141715195473, 14.508766143838448, 11.529965478703382, 9.331571854983013, 10.072757682450764, 11.849816668043786, 9.9468698979479, 10.905770223456846]
10-fold R2s: [0.7620738777080501, 0.7735331241511938, 0.7324317994545082, 0.6537559674220361, 0.73282702340532, 0.7650059129183924, 0.7598573898306157, 0.7176781668826797, 0.7482175895492376, 0.7605563789794861]
Mean RMSE: 10.770935495145803
Mean R2: 0.740593723030152


## predict the SOCD in mountain areas by random forest

In [7]:
mountain_mask = ee.Image('projects/socd-liuziyan/assets/global_mountain_classification_UNEP')
kernel = ee.Kernel.circle(radius=2500, units='meters', normalize=True)
dilated = mountain_mask.focal_max(kernel=kernel, iterations=1)
closed_mask = dilated.focal_min(kernel=kernel, iterations=1)

embedding_2024_masked = embedding_2024.updateMask(closed_mask)
orgc_prediction = embedding_2024_masked.classify(optimal_model)
orgc_prediction = orgc_prediction.rename('socd_100cm_prediction')

In [8]:
Map = geemap.Map(center=[20,0], zoom=2)
prediction_vis = {
    'min': 0,
    'max': 50,
    'palette': [    
        '#543005', '#bc8120', '#dfc27d', '#f6e8c3',
        '#f5f5f5', '#c7eae5', '#80cdc1', '#35978f',
        '#01665e', '#003c30'
    ]
}
Map.addLayer(orgc_prediction, prediction_vis, 'SOCD Prediction')

In [9]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

In [10]:
orgc_prediction_global = embedding_2024.classify(optimal_model)
Map.addLayer(orgc_prediction_global, prediction_vis, 'SOCD Prediction')

In [11]:
def create_grid(region, scale, crs, add_id=True, id_field='grid_id'):
    # Convert inputs to geometry
    if isinstance(region, ee.featurecollection.FeatureCollection) or isinstance(region, ee.feature.Feature):
        geom = region.geometry()
    elif isinstance(region, ee.geometry.Geometry):
        geom = region
    else:
        # Attempt to cast if a GeoJSON-like dict is passed
        geom = ee.Geometry(region)

    proj = ee.Projection(crs)

    # coveringGrid returns a collection of rectangular features in the given projection/scale
    grid = ee.FeatureCollection(geom.coveringGrid(proj=proj, scale=scale))

    if not add_id:
        return grid

    # Add sequential id (stable order is not guaranteed, but good enough for batch processing)
    grid_list = grid.toList(grid.size())

    def _set_id(i):
        i = ee.Number(i)
        ft = ee.Feature(grid_list.get(i))
        return ft.set(id_field, i.add(1))

    grid_with_id = ee.FeatureCollection(ee.List.sequence(0, grid.size().subtract(1)).map(_set_id))
    return grid_with_id

In [12]:
MAX_LAT = 85.05112878

region = ee.Geometry.Rectangle(
    [-180, -MAX_LAT, 180, MAX_LAT],
    proj='EPSG:4326',
    geodesic=False
)
grid = create_grid(region, scale=2000000, crs='EPSG:3857', add_id=True, id_field='GRID_ID')
Map.addLayer(grid, {}, 'Export Grid')

In [13]:
grid.size().getInfo()

462

In [39]:
export_img = orgc_prediction_global.toFloat()
drive_folder = 'GEE_socd_mountain_0_100cm_90m'

count = grid.size().getInfo()
fc_list = grid.toList(count)

print('Starting exports:', count)

for i in range(count):
    feat = ee.Feature(fc_list.get(i))

    tile_id = feat.get('GRID_ID').getInfo()
    geom = feat.geometry() 

    desc = f'socd_pred_tile_{tile_id}'
    task = ee.batch.Export.image.toDrive(
        image=export_img,
        description=desc,
        folder=drive_folder,
        fileNamePrefix=desc,
        region=geom,
        scale=90,           
        crs='EPSG:3857',   
        maxPixels=1e13,
        fileFormat='GeoTIFF'
    )
    task.start()
    print('Started task:', desc)

Starting exports: 462
Started task: socd_pred_tile_1
Started task: socd_pred_tile_2
Started task: socd_pred_tile_3
Started task: socd_pred_tile_4
Started task: socd_pred_tile_5
Started task: socd_pred_tile_6
Started task: socd_pred_tile_7
Started task: socd_pred_tile_8
Started task: socd_pred_tile_9
Started task: socd_pred_tile_10
Started task: socd_pred_tile_11
Started task: socd_pred_tile_12
Started task: socd_pred_tile_13
Started task: socd_pred_tile_14
Started task: socd_pred_tile_15
Started task: socd_pred_tile_16
Started task: socd_pred_tile_17
Started task: socd_pred_tile_18
Started task: socd_pred_tile_19
Started task: socd_pred_tile_20
Started task: socd_pred_tile_21
Started task: socd_pred_tile_22
Started task: socd_pred_tile_23
Started task: socd_pred_tile_24
Started task: socd_pred_tile_25
Started task: socd_pred_tile_26
Started task: socd_pred_tile_27
Started task: socd_pred_tile_28
Started task: socd_pred_tile_29
Started task: socd_pred_tile_30
Started task: socd_pred_til

## building the SVR and LR model for comparison

### SVR

In [73]:
svr_model = ee.Classifier.libsvm(
    svmType='EPSILON_SVR',
    kernelType= 'RBF',
    gamma= 0.1,
    cost= 10,
).setOutputMode('REGRESSION').train(
    features=training,
    classProperty='socd_100cm',
    inputProperties=band_names
)

In [75]:
def compute_metrics(samples, label_prop, model):
    preds = samples.classify(model).map(
        lambda f: f.set(
            'residual', ee.Number(f.get('classification')).subtract(f.get(label_prop)),
            'sq_residual', ee.Number(f.get('classification')).subtract(f.get(label_prop)).pow(2)
        )
    )
    mse = ee.Number(preds.reduceColumns(ee.Reducer.mean(), ['sq_residual']).get('mean'))
    rmse = mse.sqrt()

    y_mean = ee.Number(samples.reduceColumns(ee.Reducer.mean(), [label_prop]).get('mean'))
    ss_res = ee.Number(preds.reduceColumns(ee.Reducer.sum(), ['sq_residual']).get('sum'))
    ss_tot = ee.Number(
        samples.map(lambda f: f.set('sq_total', ee.Number(f.get(label_prop)).subtract(y_mean).pow(2)))
               .reduceColumns(ee.Reducer.sum(), ['sq_total']).get('sum')
    )
    r2 = ee.Number(1).subtract(ss_res.divide(ss_tot))
    return ee.Dictionary({'rmse': rmse, 'r2': r2})


training_samp = training
validation_samp = test

train_metrics = compute_metrics(training_samp, 'socd_100cm', svr_model)
val_metrics = compute_metrics(validation_samp, 'socd_100cm', svr_model)

print('Train RMSE:', ee.Number(train_metrics.get('rmse')).getInfo())
print('Train R2  :', ee.Number(train_metrics.get('r2')).getInfo())
print('Valid RMSE:', ee.Number(val_metrics.get('rmse')).getInfo())
print('Valid R2  :', ee.Number(val_metrics.get('r2')).getInfo())

Train RMSE: 17.91215026059989
Train R2  : 0.3115119344816436
Valid RMSE: 15.554739221672287
Valid R2  : 0.3644332069428956


### linear regression

In [ ]:
band_names_const = band_names.add('constant')
training_lm = training.map(lambda f: f.set('constant', 1))
lm = training_lm.reduceColumns(
    reducer=ee.Reducer.linearRegression(band_names_const.size(), 1),
    selectors=band_names_const.cat(ee.List(['socd_100cm']))
)
coeffs = ee.Array(lm.get('coefficients'))
coef_img = ee.Image(coeffs).arrayProject([0]).arrayFlatten([band_names_const])


linear_pred_img = (embedding_2024
                   .addBands(ee.Image(1).rename('constant'))
                   .select(band_names_const)
                   .multiply(coef_img)
                   .reduce(ee.Reducer.sum())
                   .rename('yhat'))


lin_train_preds = linear_pred_img.sampleRegions(
    collection=ee.FeatureCollection(training_fc),
    properties=['socd_100cm'],
    scale=10, tileScale=16
)
lin_val_preds = linear_pred_img.sampleRegions(
    collection=ee.FeatureCollection(validation_fc),
    properties=['socd_100cm'],
    scale=10, tileScale=16
)


def calc_metrics(fc, label_prop, pred_prop):
    fc = ee.FeatureCollection(fc).map(
        lambda f: f.set(
            'residual', ee.Number(f.get(pred_prop)).subtract(f.get(label_prop)),
            'sq_residual', ee.Number(f.get(pred_prop)).subtract(f.get(label_prop)).pow(2)
        )
    )
    mse = ee.Number(fc.reduceColumns(ee.Reducer.mean(), ['sq_residual']).get('mean'))
    rmse = mse.sqrt()
    y_mean = ee.Number(fc.reduceColumns(ee.Reducer.mean(), [label_prop]).get('mean'))
    ss_res = ee.Number(fc.reduceColumns(ee.Reducer.sum(), ['sq_residual']).get('sum'))
    ss_tot = ee.Number(
        fc.map(lambda f: f.set('sq_total', ee.Number(f.get(label_prop)).subtract(y_mean).pow(2)))
          .reduceColumns(ee.Reducer.sum(), ['sq_total']).get('sum')
    )
    r2 = ee.Number(1).subtract(ss_res.divide(ss_tot))
    return ee.Dictionary({'rmse': rmse, 'r2': r2})


train_metrics = calc_metrics(lin_train_preds, 'socd_100cm', 'yhat')
val_metrics   = calc_metrics(lin_val_preds,  'socd_100cm', 'yhat')

print('Linear Train RMSE:', ee.Number(train_metrics.get('rmse')).getInfo())
print('Linear Train R2  :', ee.Number(train_metrics.get('r2')).getInfo())
print('Linear Valid RMSE:', ee.Number(val_metrics.get('rmse')).getInfo())
print('Linear Valid R2  :', ee.Number(val_metrics.get('r2')).getInfo())

Linear Train RMSE: 17.28847050021146
Linear Train R2  : 0.3586219168601882
Linear Valid RMSE: 15.134872342414297
Linear Valid R2  : 0.3982816531968152


## calculating the uncertainty

In [14]:
# ===== 正式版参数 =====
n_boot = 30           # 正式版自助抽样次数（可根据资源/时间调整）
test_scale = 90        # 最终输出分辨率（m）
# 生产版在全域/大区上运行，去掉仅测试用的小 ROI。如果确实需要局部运行，可重新定义 roi。
# roi = ee.Geometry.Rectangle([100.0, 34.0, 101.0, 35.0], proj='EPSG:4326', geodesic=False)

seeds = ee.List.sequence(1, n_boot)

def train_and_predict(seed):
    seed = ee.Number(seed)

    model = ee.Classifier.smileRandomForest(
        numberOfTrees=160,        # 与之前调优结果一致或按需调整
        variablesPerSplit=8,
        minLeafPopulation=5,
        bagFraction=0.8,
        seed=seed
    ).setOutputMode('REGRESSION').train(
        features=training,
        classProperty='socd_100cm',
        inputProperties=band_names
    )

    pred = (embedding_2024
            .classify(model)
            .select('classification')
            .rename('pred')
            # 正式版在最终尺度上统一投影（这里用 EPSG:4326 保持与 embedding 一致）
            .reproject('EPSG:4326', None, test_scale)
            .toFloat())
    return pred

pred_ic = ee.ImageCollection.fromImages(seeds.map(train_and_predict))

boot_mean = pred_ic.mean().rename('boot_mean')
boot_std  = pred_ic.reduce(ee.Reducer.stdDev()).rename('boot_std')
boot_cv   = boot_std.divide(boot_mean.abs().max(1e-6)).rename('boot_cv')

# ===== 可视化（正式版图层名） =====
# Map.centerObject(embedding_2024.geometry(), 3)
# Map.addLayer(boot_mean, prediction_vis, 'bootstrap_mean_FINAL')
# Map.addLayer(boot_std,  {'min':0, 'max':10, 'palette': ['#f7fbff', '#deebf7', '#9ecae1', '#3182bd', '#08519c']}, 'bootstrap_std_FINAL')
# Map.addLayer(boot_cv,   {'min':0, 'max':1, 'palette': ['#ffffff','#fee0b6','#fdb863','#e6582a','#b30000']}, 'bootstrap_cv_FINAL')

In [61]:
Map.addLayer(boot_std, {'min':0, 'max':2, 'palette': ['#f7fbff', '#deebf7', '#9ecae1', '#3182bd', '#08519c']}, 'bootstrap_std_TEST')
Map

Map(bottom=104806.0, center=[34.24132422972856, 100.5853271484375], controls=(WidgetControl(options=['position…

In [73]:
export_img = boot_std.toFloat()
drive_folder = 'GEE_socd_mountain_std_0_100cm_90m'

count = grid.size().getInfo()
fc_list = grid.toList(count)

print('Starting exports:', count)

for i in range(count):
    feat = ee.Feature(fc_list.get(i))

    tile_id = feat.get('GRID_ID').getInfo()
    geom = feat.geometry() 

    desc = f'socd_boot_std_tile_{tile_id}'
    task = ee.batch.Export.image.toDrive(
        image=export_img,
        description=desc,
        folder=drive_folder,
        fileNamePrefix=desc,
        region=geom,
        scale=90,           
        crs='EPSG:3857',   
        maxPixels=1e13,
        fileFormat='GeoTIFF'
    )
    task.start()
    print('Started task:', desc)

Starting exports: 462
Started task: socd_boot_std_tile_1
Started task: socd_boot_std_tile_2
Started task: socd_boot_std_tile_3
Started task: socd_boot_std_tile_4
Started task: socd_boot_std_tile_5
Started task: socd_boot_std_tile_6
Started task: socd_boot_std_tile_7
Started task: socd_boot_std_tile_8
Started task: socd_boot_std_tile_9
Started task: socd_boot_std_tile_10
Started task: socd_boot_std_tile_11
Started task: socd_boot_std_tile_12
Started task: socd_boot_std_tile_13
Started task: socd_boot_std_tile_14
Started task: socd_boot_std_tile_15
Started task: socd_boot_std_tile_16
Started task: socd_boot_std_tile_17
Started task: socd_boot_std_tile_18
Started task: socd_boot_std_tile_19
Started task: socd_boot_std_tile_20
Started task: socd_boot_std_tile_21
Started task: socd_boot_std_tile_22
Started task: socd_boot_std_tile_23
Started task: socd_boot_std_tile_24
Started task: socd_boot_std_tile_25
Started task: socd_boot_std_tile_26
Started task: socd_boot_std_tile_27
Started task: s

In [15]:
export_img = boot_std.toFloat()
drive_folder = 'GEE_socd_mountain_std_0_100cm_90m'

missing_ids = [1,2,3,4,5,6,13,15,16,18,19,21,23,25,27,28,29,31,32,33,34,36,
               40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,
               61,62,63,64,66,67,68,69,70,71,73,74,75,76,77,78,79,80,81,82,83,
               84,85,86,87,88,89,91,93,94,105,106,112,116,117,126,134,143,155,
               217,222,224,228,249,259,284]

print('Starting exports for missing tiles:', len(missing_ids))

for tile_id in missing_ids:
    feat = ee.Feature(grid.filter(ee.Filter.eq('GRID_ID', tile_id)).first())
    if feat is None:
        print(f'Skip GRID_ID {tile_id}: not found in grid')
        continue

    geom = feat.geometry()
    desc = f'socd_boot_std_tile_{tile_id}'
    task = ee.batch.Export.image.toDrive(
        image=export_img,
        description=desc,
        folder=drive_folder,
        fileNamePrefix=desc,
        region=geom,
        scale=90,
        crs='EPSG:3857',
        maxPixels=1e13,
        fileFormat='GeoTIFF'
    )
    task.start()
    print('Started task:', desc)

Starting exports for missing tiles: 89
Started task: socd_boot_std_tile_1
Started task: socd_boot_std_tile_2
Started task: socd_boot_std_tile_3
Started task: socd_boot_std_tile_4
Started task: socd_boot_std_tile_5
Started task: socd_boot_std_tile_6
Started task: socd_boot_std_tile_13
Started task: socd_boot_std_tile_15
Started task: socd_boot_std_tile_16
Started task: socd_boot_std_tile_18
Started task: socd_boot_std_tile_19
Started task: socd_boot_std_tile_21
Started task: socd_boot_std_tile_23
Started task: socd_boot_std_tile_25
Started task: socd_boot_std_tile_27
Started task: socd_boot_std_tile_28
Started task: socd_boot_std_tile_29
Started task: socd_boot_std_tile_31
Started task: socd_boot_std_tile_32
Started task: socd_boot_std_tile_33
Started task: socd_boot_std_tile_34
Started task: socd_boot_std_tile_36
Started task: socd_boot_std_tile_40
Started task: socd_boot_std_tile_41
Started task: socd_boot_std_tile_42
Started task: socd_boot_std_tile_43
Started task: socd_boot_std_til